In [1]:
import cv2
import numpy as np
import csv
import time


colors = {
    'yellow': ([20, 100, 100], [30, 255, 255]),
    'white': ([0, 0, 200], [180, 30, 255]),
    'orange': ([10, 100, 100], [20, 255, 255]),
    'blue': ([90, 100, 100], [130, 255, 255])
}


frame_width = 800 
frame_height = 600  

def set_quadrants(width, height):
    """Set quadrant boundaries based on frame dimensions."""
    return {
        3: (0, 0, width // 2, height // 2),          # Top-left
        4: (width // 2, 0, width // 2, height // 2),  # Top-right
        2: (0, height // 2, width // 2, height // 2), # Bottom-left
        1: (width // 2, height // 2, width // 2, height // 2)  # Bottom-right
    }

# Track ball positions to detect entry and exit events
ball_positions = {color: None for color in colors.keys()}

def detect_balls(frame, frame_time, csv_writer, quadrants):
    """
    Detects colored balls in the frame and logs their entry or exit from defined quadrants.

    Args:
    - frame: Input frame from the video
    - frame_time: Time of the current frame in the video
    - csv_writer: CSV writer object for logging events
    - quadrants: Dictionary defining quadrant boundaries

    Returns:
    - frame: Processed frame with visual annotations
    """
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    for color_name, (lower_color, upper_color) in colors.items():
        lower_color = np.array(lower_color)
        upper_color = np.array(upper_color)
        mask = cv2.inRange(hsv_frame, lower_color, upper_color)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 100:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x + w, y + h), [int(c) for c in upper_color], 2)
                
                ball_center_x = x + w // 2
                ball_center_y = y + h // 2
                
                current_quadrant = None
                for quadrant_num, (qx, qy, qw, qh) in quadrants.items():
                    if qx <= ball_center_x < qx + qw and qy <= ball_center_y < qy + qh:
                        current_quadrant = quadrant_num
                        break
                
                if ball_positions[color_name] is None:
                    ball_positions[color_name] = current_quadrant
                
                previous_quadrant = ball_positions[color_name]
                if current_quadrant != previous_quadrant:
                    if previous_quadrant is not None:
                        event_type = "Exit"
                        print(f"Logging: {frame_time}, {previous_quadrant}, {color_name}, {event_type}")  # Debug print
                        csv_writer.writerow([frame_time, previous_quadrant, color_name, event_type])
                        cv2.putText(frame, f"Exit: {frame_time:.2f}", (ball_center_x, ball_center_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [int(c) for c in upper_color], 2)
                    
                    if current_quadrant is not None:
                        event_type = "Entry"
                        print(f"Logging: {frame_time}, {current_quadrant}, {color_name}, {event_type}")  # Debug print
                        csv_writer.writerow([frame_time, current_quadrant, color_name, event_type])
                        cv2.putText(frame, f"Entry: {frame_time:.2f}", (ball_center_x, ball_center_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [int(c) for c in upper_color], 2)
                
                ball_positions[color_name] = current_quadrant
    
    return frame

def process_video(video_path):
    """
    Processes the input video to detect colored balls and track their movement across defined quadrants.
    Saves the processed video and logs events to a CSV file.

    Args:
    - video_path: Path to the input video file
    """
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    quadrants = set_quadrants(frame_width, frame_height)
    
    out_video_path = 'processed_video.mp4'
    out_video = cv2.VideoWriter(out_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
    
    event_log_path = 'event_log.csv'
    with open(event_log_path, 'w', newline='') as event_log_file:
        csv_writer = csv.writer(event_log_file)
        csv_writer.writerow(['Time', 'Quadrant Number', 'Ball Colour', 'Event Type'])
        
        start_time = time.time()
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            current_time = time.time()
            frame_time = current_time - start_time
            
            processed_frame = detect_balls(frame, frame_time, csv_writer, quadrants)
            out_video.write(processed_frame)
            
            cv2.imshow('Processed Video', processed_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    out_video.release()
    cv2.destroyAllWindows()
    
    print(f"Processed video saved as: {out_video_path}")
    print(f"Event log saved as: {event_log_path}")

if __name__ == "__main__":
    video_path = r"C:\Users\SHANMUKH\Downloads\AI Assignment video.mp4"
    process_video(video_path)


Logging: 0.05829453468322754, 1, white, Exit
Logging: 0.05829453468322754, 4, white, Entry
Logging: 0.05829453468322754, 1, blue, Exit
Logging: 0.05829453468322754, 4, blue, Entry
Logging: 0.25738978385925293, 4, white, Exit
Logging: 0.25738978385925293, 1, white, Entry
Logging: 0.25738978385925293, 1, white, Exit
Logging: 0.25738978385925293, 4, white, Entry
Logging: 0.25738978385925293, 4, white, Exit
Logging: 0.25738978385925293, 3, white, Entry
Logging: 0.25738978385925293, 3, white, Exit
Logging: 0.25738978385925293, 4, white, Entry
Logging: 0.25738978385925293, 4, blue, Exit
Logging: 0.25738978385925293, 1, blue, Entry
Logging: 0.25738978385925293, 1, blue, Exit
Logging: 0.25738978385925293, 4, blue, Entry
Logging: 0.3502371311187744, 4, white, Exit
Logging: 0.3502371311187744, 1, white, Entry
Logging: 0.3502371311187744, 1, white, Exit
Logging: 0.3502371311187744, 4, white, Entry
Logging: 0.3502371311187744, 4, blue, Exit
Logging: 0.3502371311187744, 1, blue, Entry
Logging: 0.35